In [1]:
from alpha_vantage.cryptocurrencies import CryptoCurrencies
import pandas_ta as ta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import os

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

key = os.environ['av_key']

In [2]:
cc = CryptoCurrencies(key=key, output_format='pandas')

In [3]:
df, meta_data = cc.get_crypto_intraday('ETH', market='USD', interval='1min', outputsize='full')

In [4]:
df = df.rename(columns={"4. close": "close",
                   "1. open": "open",
                   "2. high": "high",
                   "3. low": "low",
                   "5. volume": "volume"})

df = df[['open', 'high', 'low', 'close', 'volume']]

In [5]:
df = df.sort_index()
df['idx_int'] = np.arange(0, len(df))
df = df.reset_index()

In [6]:
df.head()

,date,open,high,low,close,volume,idx_int
0,2024-11-09 13:07:00,3038.87,3039.28,3037.96,3038.27,14.0,0
1,2024-11-09 13:08:00,3038.27,3040.15,3038.03,3039.95,32.0,1
2,2024-11-09 13:09:00,3039.95,3040.06,3038.81,3038.94,20.0,2
3,2024-11-09 13:10:00,3038.97,3039.60,3036.32,3036.53,23.0,3
4,2024-11-09 13:11:00,3036.47,3037.50,3035.35,3036.38,19.0,4


In [7]:
def compute_gradient(start_index, df, x_label, y_label, len_data):
    # Ensure we only take data points from n to n+5
    if start_index + len_data > len(df):
        return None  # Return None if there are not enough points to calculate gradient
    
    # Extract the last 5 data points (x, y) from the DataFrame
    data_segment = df.iloc[start_index:start_index + len_data]
    x = data_segment[x_label]
    y = data_segment[y_label]
    
    # Calculate the necessary summations for the least squares formula
    n = len(x)
    sum_x = x.sum()
    sum_y = y.sum()
    sum_x2 = (x**2).sum()
    sum_xy = (x * y).sum()
    
    # Calculate the slope (gradient) using the least squares formula
    slope = (n * sum_xy - sum_x * sum_y) / (n * sum_x2 - sum_x**2)
    return slope

def check_crossing(df, col1, col2):
    # Calculate the difference between the two columns
    diff = df[col1] - df[col2]
    diff = diff/np.abs(diff)
    # Check if there is a sign change in the difference
    crossing = ((diff.shift(1) * diff) - 1)/-2
    
    return crossing

In [8]:
# realtime stock trading
df.ta.ema(length=10, append=True)
df.ta.ema(length=50, append=True)
df.ta.rsi(length=14, append=True)
df.ta.bbands(length=20, std=2, append=True)
df.ta.macd(fast=12, slow=26, signal=9, append=True)
df.ta.psar(append=True)

,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2
0,NaN,NaN,0.02,0
1,3037.96,NaN,0.04,0
2,3037.96,NaN,0.04,0
3,NaN,3040.150000,0.02,1
4,NaN,3040.073400,0.04,0
...,...,...,...,...
345,NaN,3049.390112,0.04,0
346,NaN,3049.128908,0.04,0
347,NaN,3048.878151,0.04,0
348,NaN,3048.637425,0.04,0


In [9]:
try: df = df.rename(columns={'EMA_60':'EMA_50'})
except: print("skip")

In [10]:
df['psar_flip_dir'] = 0
df.loc[(df['PSARr_0.02_0.2']==1) & (df['PSARl_0.02_0.2'].isnull()==False), 'psar_flip_dir'] = 1
df.loc[(df['PSARr_0.02_0.2']==1) & (df['PSARs_0.02_0.2'].isnull()==False), 'psar_flip_dir'] = -1

In [11]:
df.head()

,date,open,high,low,close,volume,idx_int,EMA_10,EMA_50,RSI_14,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2,psar_flip_dir
0,2024-11-09 13:07:00,3038.87,3039.28,3037.96,3038.27,14.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.02,0,0
1,2024-11-09 13:08:00,3038.27,3040.15,3038.03,3039.95,32.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3037.96,NaN,0.04,0,0
2,2024-11-09 13:09:00,3039.95,3040.06,3038.81,3038.94,20.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3037.96,NaN,0.04,0,0
3,2024-11-09 13:10:00,3038.97,3039.60,3036.32,3036.53,23.0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3040.1500,0.02,1,-1
4,2024-11-09 13:11:00,3036.47,3037.50,3035.35,3036.38,19.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3040.0734,0.04,0,0


In [12]:
df['r_ema_s_m'] = df['EMA_10'] / df['EMA_50']

In [13]:
len_data = 3

# Loop through the DataFrame, compute gradient for each row (starting point)
for i in range(len(df) - len_data):  # Make sure we have at least 5 points for each calculation
    gradient = compute_gradient(i, df, 'idx_int', 'EMA_10', len_data)
    df.at[i + len_data-1, 'gradient_ema_10'] = gradient  # Store the gradient in the row corresponding to n+4
    gradient = compute_gradient(i, df, 'idx_int', 'RSI_14', len_data)
    df.at[i + len_data-1, 'gradient_rsi_14'] = gradient  # Store the gradient in the row corresponding to n+4
    gradient = compute_gradient(i, df, 'idx_int', 'close', len_data)
    df.at[i + len_data-1, 'gradient_ls'] = gradient
    gradient = compute_gradient(i, df, 'idx_int', 'EMA_50', len_data)
    df.at[i + len_data-1, 'gradient_ema_50'] = gradient  # Store the gradient in the row corresponding to n+4

In [14]:
# mask_crossing = (df['r_ema_s_m']<1.0005)&(df['r_ema_s_m']>0.9995)
# df['flag_ema_crossing'] = 0
# df.loc[mask_crossing, 'flag_ema_crossing'] = 1

In [15]:
df['flag_ema_crossing'] = check_crossing(df, 'EMA_10', 'EMA_50')

In [16]:
df.iloc[-20:]

,date,open,high,low,close,volume,idx_int,EMA_10,EMA_50,RSI_14,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2,psar_flip_dir,r_ema_s_m,gradient_ema_10,gradient_rsi_14,gradient_ls,gradient_ema_50,flag_ema_crossing
330,2024-11-09 18:37:00,3047.45,3047.70,3045.36,3045.72,81.0,330,3046.047204,3038.220581,59.922514,3038.789096,3044.1275,3049.465904,0.350735,0.649155,2.858562,-0.156051,3.014612,3043.293148,NaN,0.12,0,0,1.002576,0.111431,-4.850241,-1.200,0.347652,-0.0
331,2024-11-09 18:38:00,3045.36,3046.35,3044.49,3045.21,18.0,331,3045.894985,3038.494675,58.034447,3038.973523,3044.2690,3049.564477,0.347898,0.588849,2.609400,-0.324170,2.933570,3043.946770,NaN,0.12,0,0,1.002436,-0.112466,-4.364908,-1.120,0.290097,-0.0
332,2024-11-09 18:39:00,3045.13,3046.61,3045.13,3046.08,11.0,332,3045.928624,3038.792139,60.330680,3039.730649,3044.5985,3049.466351,0.319770,0.652172,2.453852,-0.383774,2.837626,3044.490000,NaN,0.12,0,0,1.002348,-0.059290,0.204083,0.180,0.285779,-0.0
333,2024-11-09 18:40:00,3045.87,3046.02,3044.32,3045.85,29.0,333,3045.914329,3039.068918,59.405255,3040.932580,3044.9620,3048.991420,0.264661,0.610190,2.285673,-0.441563,2.727236,NaN,3048.740000,0.02,1,-1,1.002252,0.009672,0.685404,0.320,0.287121,-0.0
334,2024-11-09 18:41:00,3045.56,3047.69,3045.55,3047.33,32.0,334,3046.171724,3039.392882,63.305749,3041.840466,3045.3230,3048.805534,0.228714,0.788152,2.245924,-0.385050,2.630973,NaN,3048.651600,0.02,0,0,1.002230,0.121550,1.487534,0.625,0.300371,-0.0
335,2024-11-09 18:42:00,3047.67,3048.38,3046.18,3046.72,25.0,335,3046.271410,3039.680220,60.716297,3042.006430,3045.4680,3048.929570,0.227326,0.680843,2.140525,-0.392358,2.532884,NaN,3048.564968,0.02,0,0,1.002168,0.178541,0.655521,0.435,0.305651,-0.0
336,2024-11-09 18:43:00,3046.52,3048.38,3046.19,3048.32,17.0,336,3046.643881,3040.019035,64.785095,3042.424192,3045.7705,3049.116808,0.219735,0.880942,2.161190,-0.297355,2.458545,NaN,3048.480069,0.02,0,0,1.002179,0.236079,0.739673,0.495,0.313076,-0.0
337,2024-11-09 18:44:00,3047.95,3048.38,3046.20,3046.74,48.0,337,3046.661357,3040.282602,58.357185,3042.870642,3045.9635,3049.056358,0.203079,0.625531,2.026712,-0.345466,2.372178,NaN,3048.396867,0.02,0,0,1.002098,0.194973,-1.179556,0.010,0.301191,-0.0
338,2024-11-09 18:45:00,3046.46,3048.37,3045.68,3046.97,27.0,338,3046.717474,3040.544853,58.994988,3043.210086,3046.1320,3049.053914,0.191844,0.643399,1.916603,-0.364461,2.281063,3044.320000,NaN,0.02,1,1,1.002030,0.036796,-2.895053,-0.675,0.262909,-0.0
339,2024-11-09 18:46:00,3046.92,3048.50,3046.72,3048.50,5.0,339,3047.041570,3040.856820,63.049307,3043.551946,3046.3825,3049.213054,0.185830,0.874043,1.930544,-0.280415,2.210959,3044.401000,NaN,0.04,0,0,1.002034,0.190106,2.346061,0.880,0.287109,-0.0


In [17]:
mask_ema_grad_pos = (df['gradient_ema_10']>0.05)
mask_ema_grad_neg = (df['gradient_ema_10']<-0.05)
df['flag_grad_ema'] = 0
df.loc[mask_ema_grad_pos, 'flag_grad_ema'] = 1
df.loc[mask_ema_grad_neg, 'flag_grad_ema'] = -1

mask_ema_grad_pos = (df['gradient_ema_50']>0.05)
mask_ema_grad_neg = (df['gradient_ema_50']<-0.05)
df['flag_grad_ema_50'] = 0
df.loc[mask_ema_grad_pos, 'flag_grad_ema_50'] = 1
df.loc[mask_ema_grad_neg, 'flag_grad_ema_50'] = -1

mask_rsi_grad_pos = (df['gradient_rsi_14']>=1)
mask_rsi_grad_neg = (df['gradient_rsi_14']<=1)
df['flag_grad_rsi'] = 0
df.loc[mask_rsi_grad_pos, 'flag_grad_rsi'] = 1
df.loc[mask_rsi_grad_neg, 'flag_grad_rsi'] = -1

df['flag_grad_ls'] = 0
df.loc[df['gradient_ls'] >= 0.05, 'flag_grad_ls'] = 1
df.loc[df['gradient_ls'] <= -0.05, 'flag_grad_ls'] = -1

In [18]:
df['ema_short_above_or_below'] = 0
df.loc[(df['EMA_10']>df['EMA_50']), 'ema_short_above_or_below'] = 1
df.loc[(df['EMA_10']<df['EMA_50']), 'ema_short_above_or_below'] = -1

In [19]:
df['r_close_bbu'] = df['close'] / df['BBU_20_2.0']
df['r_close_bbl'] = df['close'] / df['BBL_20_2.0']
df['r_ema_bbu'] = df['EMA_10'] / df['BBU_20_2.0']
df['r_ema_bbl'] = df['EMA_10'] / df['BBL_20_2.0']

In [20]:
df['long_entry'] = np.nan
df['short_entry'] = np.nan
df['long_exit'] = np.nan
df['short_exit'] = np.nan

In [21]:
# display(df.iloc[100:120])

# Strategy

To create a real-time trading strategy with specific technical indicator settings and strategies for entering and exiting trades, let's break down the components clearly:

### 1. **Technical Indicators**
Here's how we can set up the specific technical indicators:

#### Moving Averages
- **Short-Term Moving Average (MA)**: Use a 10-period MA. This is effective for spotting short-term trends and suitable for minute-by-minute trading data.
- **Long-Term Moving Average**: Use a 50-period MA to confirm the overall trend direction. This helps filter out the noise and provides a clearer direction.

#### Relative Strength Index (RSI)
- **Length**: A 14-period RSI is standard and works well in a variety of market conditions.
- **Overbought Threshold**: Set at 70 to indicate potential selling points.
- **Oversold Threshold**: Set at 30 to indicate potential buying points.

#### MACD (Moving Average Convergence Divergence)
- **Fast Line**: 12-period Exponential Moving Average (EMA)
- **Slow Line**: 26-period EMA
- **Signal Line**: 9-period EMA of the MACD line itself

### 2. **Entry and Exit Points**
- **Go Long (Buy)**: 
  - **Entry Point**: When the 10-period MA crosses above the 50-period MA and the RSI is below 70 but above 30, indicating neither overbought nor oversold conditions. Additionally, the MACD line should cross above the signal line.
  - **Exit Point (Take Profit)**: Consider exiting when the 10-period MA crosses back below the 50-period MA, or RSI approaches 70, indicating a potential overbought condition.
  
- **Go Short (Sell)**: 
  - **Entry Point**: When the 10-period MA crosses below the 50-period MA and the RSI is above 30 but below 70, indicating normal conditions. Also, the MACD line should cross below the signal line.
  - **Exit Point (Take Profit)**: Consider exiting when the 10-period MA crosses back above the 50-period MA, or RSI approaches 30, indicating a potential oversold condition.

### 3. **Stop Loss Criteria**
- **For Long Positions**: Place a stop loss below the most recent significant low or a fixed percentage, such as 2-3% below the entry price, depending on your risk tolerance.
- **For Short Positions**: Place a stop loss above the most recent significant high or a fixed percentage, such as 2-3% above the entry price.

### 4. **Risk Management**
- Consider the amount of capital at risk per trade, commonly 1-2% of your total trading capital to manage risk effectively.
- Use trailing stops to lock in profits as the market moves in your favor.

### 5. **Automation and Monitoring**
- Automate the strategy using a trading bot if possible, especially to handle the high-frequency nature of minute-by-minute trading.
- Regularly monitor performance and adjust parameters as necessary based on market conditions and strategy performance.

This setup gives you a robust framework for trading on a minute-by-minute basis, leveraging these technical indicators to make informed trading decisions. Adjust these settings based on backtesting results and your specific risk tolerance.

# Long Entry

In [22]:
mask_le1 = (df['ema_short_above_or_below']==1)&(df['flag_ema_crossing']==1)&(df['flag_grad_ema']>0)
mask_le2 = (df['MACDh_12_26_9']>0)
mask_le3 = (df['r_close_bbl']<=1.0005)
mask_le4 = (df['RSI_14']<70)&(df['RSI_14']>30)
mask_le5 = (df['PSARl_0.02_0.2'] < df['close']) & (df['psar_flip_dir']>0)
mask_le6 = (df['RSI_14']<40)
mask_le7 = (df['flag_grad_ema']>=0)

In [23]:
df['ema_crossing_pos'] = 0
df.loc[mask_le1, 'ema_crossing_pos'] = 1
df['macd_pos'] = 0
df.loc[mask_le2, 'macd_pos'] = 1
df['close_to_bbl'] = 0
df.loc[mask_le3, 'close_to_bbl'] = 1
df['rsi_30_to_70'] = 0
df.loc[mask_le4, 'rsi_30_to_70'] = 1
df['PSAR_bellow_close'] = 0
df.loc[mask_le5, 'PSAR_bellow_close'] = 1

In [24]:
df['long_entry'] = 0
# df.loc[(mask_le1 & mask_le4) | (mask_le5 & mask_le4 & mask_le2) | (mask_le2 & mask_le6 & mask_le3), 'long_entry'] = 1
df.loc[(mask_le1 & mask_le4) | (mask_le6 & mask_le7), 'long_entry'] = 1

In [25]:
# df['long_entry'] = df[['ema_crossing_pos', 'macd_pos', 'rsi_30_to_70', 'PSAR_bellow_close']].sum(axis=1)

In [26]:
df.loc[df['long_entry'] == 1].shape

(4, 47)

# Short Entry

In [27]:
mask_se1 = (df['ema_short_above_or_below']==-1)&(df['flag_ema_crossing']==1)
mask_se2 = (df['MACDh_12_26_9']<0)
mask_se3 = (df['r_close_bbu']>=0.9995)
mask_se4 = (df['RSI_14']>30)&(df['RSI_14']<70)
mask_se5 = (df['PSARs_0.02_0.2'] > df['close']) & (df['psar_flip_dir']<0)

In [28]:
df['ema_crossing_neg'] = 0
df.loc[mask_se1, 'ema_crossing_neg'] = 1
df['macd_neg'] = 0
df.loc[mask_se2, 'macd_neg'] = 1
df['close_to_bbu'] = 0
df.loc[mask_se3, 'close_to_bbu'] = 1
df['rsi_30_to_70'] = 0
df.loc[mask_se4, 'rsi_30_to_70'] = 1
df['PSAR_above_close'] = 0
df.loc[mask_se5, 'PSAR_above_close'] = 1

In [29]:
df['short_entry'] = 0
df.loc[mask_se1 & mask_se5 & mask_se4, 'short_entry'] = 1

In [30]:
# df['short_entry'] = df[['ema_crossing_neg', 'macd_neg', 'rsi_30_to_70', 'PSAR_above_close']].sum(axis=1)

In [31]:
df.loc[df['short_entry']==4].shape

(0, 51)

# Long Exit

In [32]:
mask_lex1 = (df['ema_short_above_or_below']==-1)&(df['flag_ema_crossing']==1)
mask_lex2 = (df['RSI_14']>55)
mask_lex3 = (df['psar_flip_dir'] == -1)
mask_lex4 = (df['flag_grad_ema']<0)

In [33]:
df['ema_crossing_neg'] = 0
df.loc[mask_lex1, 'ema_crossing_neg'] = 1
df['rsi_above_70'] = 0
df.loc[mask_lex2, 'rsi_above_70'] = 1
df['psar_flip_neg'] = 0
df.loc[mask_lex3, 'psar_flip_neg'] = 1

In [34]:
df['long_exit'] = 0
df.loc[(mask_lex1) | (mask_lex2 & mask_lex4), 'long_exit'] = 1

In [35]:
# df['long_exit'] = df[['ema_crossing_neg', 'rsi_above_70']].sum(axis=1)

In [36]:
df.loc[df['long_exit'] == 1].shape

(9, 53)

# Short Exit

In [37]:
mask_sex1 = (df['ema_short_above_or_below']==1)&(df['flag_ema_crossing']==1)
mask_sex2 = (df['RSI_14']<=30)
mask_sex3 = (df['psar_flip_dir'] == 1)

In [38]:
df['ema_crossing_pos'] = 0
df.loc[mask_sex1, 'ema_crossing_pos'] = 1
df['rsi_bellow_30'] = 0
df.loc[mask_sex2, 'rsi_bellow_30'] = 1
df['psar_flip_pos'] = 0
df.loc[mask_sex3, 'psar_flip_pos'] = 1

In [39]:
df['short_exit'] = 0
df.loc[mask_sex1 | mask_sex2, 'short_exit'] = 1

In [40]:
# df['short_exit'] = df[['ema_crossing_pos', 'rsi_bellow_30']].sum(axis=1)

In [41]:
# df.loc[df['short_exit'] == 1].shape

# Bullish / bearish confirmation

In [42]:
df['gradient_ema_pos'] = 0
df.loc[df['flag_grad_ema']==1, 'gradient_ema_pos'] = 1
df['gradient_ema_neg'] = 0
df.loc[df['flag_grad_ema']==-1, 'gradient_ema_neg'] = 1

In [43]:
df['bullish_confirm'] = df[['ema_crossing_pos', 'macd_pos', 'gradient_ema_pos']].sum(axis=1)
df['bearish_confirm'] = df[['ema_crossing_neg', 'macd_neg', 'gradient_ema_neg']].sum(axis=1)

# Oversold Marker

In [44]:
mask_os1 = (df['RSI_14']<=30)&((df['flag_grad_ls']==1)|(df['flag_grad_ema']==1))
mask_os2 = (df['r_close_bbl']<=1.0005)&((df['flag_grad_ls']==1)|(df['flag_grad_ema']==1))

In [45]:
df['rsi_bellow_30'] = 0
df.loc[mask_os1, 'rsi_bellow_30'] = 1
df['close_to_bbl'] = 0
df.loc[mask_os2, 'close_to_bbl'] = 1

In [46]:
df['oversold_confirm'] = df[['rsi_bellow_30', 'close_to_bbl']].sum(axis=1)

# Overbought Marker

In [47]:
mask_ob1 = (df['RSI_14']>=70)&((df['flag_grad_ls']==-1)|(df['flag_grad_ema']==-1))
mask_ob2 = (df['r_close_bbu']>=0.9995)&((df['flag_grad_ls']==-1)|(df['flag_grad_ema']==-1))

In [48]:
df['rsi_above_70'] = 0
df.loc[mask_ob1, 'rsi_above_70'] = 1
df['close_to_bbu'] = 0
df.loc[mask_ob2, 'close_to_bbu'] = 1

In [49]:
df['overbought_confirm'] = df[['rsi_above_70', 'close_to_bbu']].sum(axis=1)

In [50]:
df.to_clipboard()